In [2]:
!pip -q install sng4onnx


In [3]:
import tensorflow as tf, onnx2tf, sng4onnx
print("Pronto! TF:", tf.__version__)


Pronto! TF: 2.17.1


In [4]:
# Reinicie o runtime (Runtime > Restart session) e rode isto primeiro:
!pip -q install --upgrade pip
!pip -q install --prefer-binary onnx==1.16.2 tensorflow==2.17.1 onnx-graphsurgeon
!pip -q install "onnx2tf>=1.21.0" ai-edge-litert tf-keras==2.17.0

import tensorflow as tf, onnx, onnx2tf, onnx_graphsurgeon, ai_edge_litert, tf_keras
print("OK | TF:", tf.__version__)


OK | TF: 2.17.1


In [7]:
import torch, torch.nn as nn, onnx, os
from google.colab import files

# (opcional) usa onnxsim se estiver disponível; se não, segue sem
try:
    from onnxsim import simplify
    USE_SIM = True
except Exception:
    simplify = None; USE_SIM = False
print("onnxsim:", "on" if USE_SIM else "off")

IMG_SIZE = 256
CHECKPOINT = "/content/unet_best.pt"   # <-- seu arquivo

# U-Net (mesma do treino)
class ConvBlock(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_c, out_c, 3, padding=1), nn.BatchNorm2d(out_c), nn.ReLU(inplace=True),
            nn.Conv2d(out_c, out_c, 3, padding=1), nn.BatchNorm2d(out_c), nn.ReLU(inplace=True),
        )
    def forward(self, x): return self.net(x)

class UNet(nn.Module):
    def __init__(self, in_channels=3, base=32, out_channels=1):
        super().__init__()
        self.enc1 = ConvBlock(in_channels, base)
        self.enc2 = ConvBlock(base, base*2)
        self.enc3 = ConvBlock(base*2, base*4)
        self.enc4 = ConvBlock(base*4, base*8)
        self.pool = nn.MaxPool2d(2)
        self.bottleneck = ConvBlock(base*8, base*16)
        self.up4 = nn.ConvTranspose2d(base*16, base*8, 2, stride=2)
        self.dec4 = ConvBlock(base*16, base*8)
        self.up3 = nn.ConvTranspose2d(base*8, base*4, 2, stride=2)
        self.dec3 = ConvBlock(base*8, base*4)
        self.up2 = nn.ConvTranspose2d(base*4, base*2, 2, stride=2)
        self.dec2 = ConvBlock(base*4, base*2)
        self.up1 = nn.ConvTranspose2d(base*2, base, 2, stride=2)
        self.dec1 = ConvBlock(base*2, base)
        self.head = nn.Conv2d(base, out_channels, 1)
    def forward(self, x):
        e1 = self.enc1(x); p1 = self.pool(e1)
        e2 = self.enc2(p1); p2 = self.pool(e2)
        e3 = self.enc3(p2); p3 = self.pool(e3)
        e4 = self.enc4(p3); p4 = self.pool(e4)
        b = self.bottleneck(p4)
        d4 = self.up4(b); d4 = torch.cat([d4, e4], dim=1); d4 = self.dec4(d4)
        d3 = self.up3(d4); d3 = torch.cat([d3, e3], dim=1); d3 = self.dec3(d3)
        d2 = self.up2(d3); d2 = torch.cat([d2, e2], dim=1); d2 = self.dec2(d2)
        d1 = self.up1(d2); d1 = torch.cat([d1, e1], dim=1); d1 = self.dec1(d1)
        return self.head(d1)

# 1) carregar checkpoint
model = UNet().eval()
model.load_state_dict(torch.load(CHECKPOINT, map_location="cpu"))

# 2) exportar para ONNX
dummy = torch.randn(1,3,IMG_SIZE,IMG_SIZE)
torch.onnx.export(model, dummy, "unet.onnx",
                  input_names=["input"], output_names=["logits"],
                  opset_version=17, do_constant_folding=True)

# 3) (opcional) simplificar
if USE_SIM:
    m = onnx.load("unet.onnx")
    m_simp, ok = simplify(m)
    assert ok, "Falha ao simplificar ONNX"
    onnx.save(m_simp, "unet_simp.onnx")
else:
    os.replace("unet.onnx", "unet_simp.onnx")

# 4) converter para TF + TFLite FP32
from onnx2tf import convert
convert(
    input_onnx_file_path="unet_simp.onnx",
    output_folder_path="unet_tf",
    output_integer_quantized_tflite=False,     # FP32
    copy_onnx_input_output_names_to_tflite=True,
)

# 5) baixar o .tflite
files.download("unet_tf/model_float32.tflite")


onnxsim: off

Model optimizing started ============================================================
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/onnx2tf/onnx2tf.py", line 684, in convert
    result = subprocess.check_output(
             ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/subprocess.py", line 466, in check_output
    return run(*popenargs, stdout=PIPE, timeout=timeout, check=True,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "/usr/lib/python3.12/subprocess.py", line 1955, in _execute_child
    raise child_exception_type(errno_num, err_msg, err_filename)
FileNotFoundError: [Errno 2] No such file or directory: 'onnxsim'


Automa

FileNotFoundError: Cannot find file: unet_tf/model_float32.tflite

In [8]:
import os, glob
files = glob.glob("**/*.tflite", recursive=True)
print("TFLite encontrados:", files)


TFLite encontrados: ['unet_tf/unet_simp_float16.tflite', 'unet_tf/unet_simp_float32.tflite']


In [10]:
from google.colab import files
files.download("unet_tf/unet_simp_float32.tflite")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>